<a href="https://colab.research.google.com/github/ElnathanTiokou/Adversarial_Robustness_of_FL_systems/blob/main/Tensorflow_federated_Robustness/attackers/model_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

!pip install --quiet  tensorflow-federated==0.20.0
# The latest version of tensorflow-federated is not working with the colab python version
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

     |████████████████████████████████| 819 kB 4.8 MB/s 
     |████████████████████████████████| 237 kB 60.5 MB/s 
     |████████████████████████████████| 65.1 MB 89 kB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 121 kB 70.9 MB/s 
     |████████████████████████████████| 251 kB 70.7 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 4.0 MB 39.9 MB/s 
     |████████████████████████████████| 887 kB 58.2 MB/s 
     |████████████████████████████████| 462 kB 49.4 MB/s 
     |████████████████████████████████| 4.2 MB 41.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.28.1 which is incompatible.
pymc3 3.11.4 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.
panel 0.1

In [1]:
from typing import Callable, Tuple
import abc
from typing import List, Tuple

import os
import tensorflow as tf
import numpy as np

!pip install import-ipynb
import import_ipynb
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:

# Copy the link and remove the front part of the link (i.e. https://drive.google.com/open?id=) to get the file ID.
your_module = drive.CreateFile({'id':'1uh4b-cAKF0ZORGh7af9SOSYVToyo5LeF'})  
your_module.GetContentFile('constants.ipynb')
your_module = drive.CreateFile({'id':'1CbGQgBT6dw0km4H1RbxMUS1maXIPPl9T'})  
your_module.GetContentFile('executable.ipynb')

In [4]:
import constants
import executable

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff


In [9]:
from abc import abstractmethod
from typing import List, Tuple

import numpy as np
from abc import ABC
from typing import Set

In [10]:



class AbstractAttacker(ABC):

    def __init__(self, fraction: float) -> None:
        self.fraction = fraction
        self.attacked_clients = set()

    def get_attacked_clients(self) -> Set[int]:
        return self.attacked_clients


class AbstractModelAttacker(AbstractAttacker):

    def __init__(self, fraction: float) -> None:
        super().__init__(fraction)
        self.first_trainable_layer = 0

    @abc.abstractmethod
    def forward_pass(self, dataset: tf.data.Dataset, server_model: tf.keras.Model):
        pass

    def set_first_trainable_layer(self, first_trainable_layer):
        self.first_trainable_layer = first_trainable_layer


class NoModelAttacker(AbstractModelAttacker):

    def __init__(self) -> None:
        super().__init__(0)
        self.model_fn = get_model
        self.optimizer = get_optimizer()
        self.loss = get_loss()

    def forward_pass(self, dataset: tf.data.Dataset, server_model: tf.keras.Model):
        local_model = self.model_fn()
        old_weights = server_model.get_weights()
        local_model.set_weights(old_weights)
        local_model.compile(
            optimizer=self.optimizer,
            loss=self.loss
        )
        local_model.fit(dataset, epochs=NUM_EPOCHS, verbose=0)
        weights_delta = tf.nest.map_structure(tf.subtract, local_model.get_weights()[self.first_trainable_layer:],
                                              old_weights[self.first_trainable_layer:])
        return weights_delta


class RandomModelAttacker(AbstractModelAttacker):

    def __init__(self, fraction: float, std: float) -> None:
        super().__init__(fraction)
        self.std = std
        self.attacked_clients = set(pick_clients(fraction))

    def forward_pass(self, dataset: tf.data.Dataset, server_model: tf.keras.Model):
        return [np.random.normal(0, self.std, layer_weight.shape)
                for layer_weight in server_model.get_weights()[self.first_trainable_layer:]]


class SignFlipModelAttacker(NoModelAttacker):

    def __init__(self, fraction: float, multiplier: float) -> None:
        super().__init__()
        self.fraction = fraction
        self.multiplier = multiplier
        self.attacked_clients = set(pick_clients(fraction))

    def forward_pass(self, dataset: tf.data.Dataset, server_model: tf.keras.Model):
        clean_grads = super(SignFlipModelAttacker, self).forward_pass(dataset, server_model)
        return [layer * -1 * self.multiplier for layer in clean_grads]


class BackdoorAttack(NoModelAttacker):

    def __init__(self, fraction: float, target_class: int) -> None:
        super().__init__()
        self.fraction = fraction
        self.target_class = target_class
        self.pattern_relative_size = 0.15
        self.margin_relative_size = 0.05
        self.backdoor_test_samples = []
        self.chosen_attackers = 0
        self.x_test = []
        self.y_test = []

    def attack_train(self, partitioned_data: List[Tuple[np.ndarray, np.ndarray]]) -> \
            List[Tuple[np.ndarray, np.ndarray]]:
        print(self.__class__.__name__ + " started.")
        self.attacked_clients = pick_clients(self.fraction)
        for client in self.attacked_clients:
            x_train, y_train = partitioned_data[client]
            attacked_indices = self.pick_attacked_samples(y_train)
            y_train[attacked_indices] = self.target_class
            x_train[attacked_indices] = self.add_pattern((x_train[attacked_indices]))
            partitioned_data[client] = (x_train, y_train)
        self.attacked_clients = set(self.attacked_clients)
        print(self.__class__.__name__ + " finished.")
        return partitioned_data

    def attack_test(self, x_test, y_test):
        attacked_indices = self.pick_attacked_samples(y_test)
        y_test[attacked_indices] = self.target_class
        x_test[attacked_indices] = self.add_pattern((x_test[attacked_indices]))
        self.x_test = x_test[attacked_indices]
        self.y_test = y_test[attacked_indices]
        return np.delete(x_test, attacked_indices, 0), np.delete(y_test, attacked_indices, 0)

    def add_pattern(self, x: np.ndarray):
        height, width, channel = x.shape[1:]
        pattern_width = int(np.round(self.pattern_relative_size * width))
        pattern_height = int(np.round(self.pattern_relative_size * height))
        left_margin = int(np.round(self.margin_relative_size * width))
        top_margin = int(np.round(self.margin_relative_size * height))
        slope = pattern_width / pattern_height
        x[:, [top_margin, top_margin + pattern_height], left_margin:left_margin + pattern_width + 1, :] = 255
        for image in x:
            for h in range(pattern_height):
                w = pattern_width - int(np.round(h / slope))
                image[top_margin + h, left_margin + w] = 255
        return x

    def pick_attacked_samples(self, labels: np.ndarray):

        pool = np.where(labels != self.target_class)[0]
        return np.random.choice(pool, min(int(np.round(.05 * len(labels))), len(pool)), False)

    def forward_pass(self, dataset: tf.data.Dataset, server_model: tf.keras.Model):
        weights_delta = super().forward_pass(dataset, server_model)

        # for epoch in range(NUM_EPOCHS):
        #     for x_batch, y_batch in dataset:
        #         with tf.GradientTape() as tape:
        #             out = local_model(x_batch, training=True)
        #             loss_val = self.loss(y_batch, out)
        #         grads = tape.gradient(loss_val, local_model.trainable_weights)
        #         self.optimizer.apply_gradients(zip(grads, local_model.trainable_weights))

        weights_delta = tf.nest.map_structure(lambda x: x * 10 / self.chosen_attackers, weights_delta)
        return weights_delta